In [911]:
import pdfplumber
import autogen
from autogen import ConversableAgent
import os
from autogen import GroupChat
from autogen import GroupChatManager

In [912]:
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

# pdf_text = extract_text_from_pdf("mock_trial.pdf")
pdf_text = extract_text_from_pdf("./Mini-Mock-Trial-State-v.-Anderson-2016.pdf")

In [913]:
def estimate_tokens(text):
    return len(text) // 4

total_tokens = estimate_tokens(pdf_text)
print(f"Estimated tokens: {total_tokens}")

Estimated tokens: 1854


In [914]:
import os

# Check if API key is loaded
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("API key loaded successfully.")
else:
    print("API key NOT found. Double-check your environment variable.")

API key loaded successfully.


In [915]:
llm_config = {"config_list": [{"model": "gpt-4o", "api_key": api_key, "temperature": 0.7}]}
# llm_config = {"config_list": [{"model": "llama3.2:latest", "api_type": "ollama", "client_host": "http://127.0.0.1:11434"}]}

In [916]:
# Ask the user to select a role
user_role = input("Would you like to role-play as the Defense Attorney or Prosecuting Attorney? Type DA or PA to confirm your selection.").strip().upper()

# Ensure valid selection
while user_role not in ["DA", "PA"]:
    user_role = input("Invalid role. Please enter DA or PA. \n\n Would you like to role-play as the Defense Attorney or Prosecuting Attorney? Type DA or PA to confirm your selection.").strip().upper()

agents = {}
human_proxy_role = ""

prosecuting_attorney_prompt = """
You are the Prosecuting Attorney in this mock trial. Your role is to present evidence and argue the case on behalf of the prosecution. Follow courtroom procedure, make legal arguments, and question witnesses to prove the defendant's guilt. 
When asking questions to witnesses or the defendant, only ask one question at a time and end your turn.
"""

defense_attorney_prompt = """
You are the Defense Attorney in this mock trial. Your role is to defend the defendant by creating reasonable doubt and presenting legal arguments that support their innocence. Cross-examine witnesses and challenge the prosecution's claims.
When asking questions to witnesses or the defendant, only ask one question at a time and end your turn.
"""

if user_role == "DA":
    human_proxy_role = "defense attorney"

    agents["prosecuting_attorney"] = ConversableAgent(
        "prosecuting_attorney",
        system_message=prosecuting_attorney_prompt,
        llm_config=llm_config,
        description="""
        Acts as the Prosecuting Attorney in a mock trial. 
        Will always wait for the Defense Attorney to explicitly say they are finished with questioning before speaking.
        Responsible for presenting the case against the defendant, introducing evidence, questioning witnesses, and making legal arguments that establish the defendant's guilt under the applicable law.
        Should not speak until the Defense Attorney has finished their turn and explicity says they are finished with questioning.
        The Prosecuting Attorney seeks to achieve a conviction through clear and compelling arguments based on the trial's evidence.
        """
    )

    agents["human_proxy"] = ConversableAgent(
        "human_proxy-defense_attorney",
        llm_config=False,
        human_input_mode="NEVER",
        is_termination_msg=lambda message: True,
        description="""
        Acts as the Defense Attorney in a mock trial. Should be prompted to speak when the defendant is being questioned.
        Responsible for representing the defendant by crafting a defense strategy, challenging the prosecution's case, cross-examining witnesses, and presenting evidence that supports the defendant's innocence or mitigates their liability.
        """
    )
else:
    human_proxy_role = "prosecuting attorney"
    
    agents["defense_attorney"] = ConversableAgent(
        "defense_attorney",
        system_message=defense_attorney_prompt,
        llm_config=llm_config,
        description="""
        Acts as the Defense Attorney in a mock trial. 
        Will always wait for the Prosecuting Attorney to explicitly say they are finished with questioning before speaking.
        Responsible for representing the defendant, delivering legal arguments, challenging the prosecution's claims, and advocating for the most favorable outcome for their client through legal defense strategies and courtroom procedure.
        Should not speak until the Prosecuting Attorney has finished their turn and explicity says they are finished with questioning.
        """
    )

    agents["human_proxy"] = ConversableAgent(
        "human_proxy-prosecuting_attorney",
        llm_config=False,
        human_input_mode="NEVER",
        is_termination_msg=lambda message: True,
        description="""
        Acts as the Prosecuting Attorney in a mock trial. 
        Responsible for presenting the case against the defendant, introducing evidence, questioning witnesses, and making legal arguments that establish the defendant's guilt under the applicable law.
        The Prosecuting Attorney seeks to achieve a conviction through clear and compelling arguments based on the trial's evidence.
        """
    )

In [917]:
agents["judge_agent"] = ConversableAgent(
    "judge_agent", 
    system_message="""
    You are a judge presiding over a mock trial. 
    Answer all questions **only from the judge's perspective**, making legal rulings, managing courtroom procedures, and ensuring a fair trial. 
    Do not simulate conversations between other trial participants or generate responses for other roles. Focus solely on delivering rulings and courtroom instructions.
    """, 
    llm_config=llm_config,  
    description="""
    Presides over the mock trial as the judge. 
    Responsible for ensuring the trial proceeds fairly and in accordance with legal principles. 
    Duties include interpreting the law, making procedural rulings, managing courtroom conduct, 
    overseeing the admissibility of evidence, ruling on objections, and instructing the jury. 
    The judge must remain impartial while facilitating the smooth progression of the trial.
    """,
    is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
)

agents["defendant_agent"] = ConversableAgent(
    name="defendant_agent", 
    system_message="""
    You are the defendant in a mock trial. 
    Answer all questions **only from the defendant's perspective**, responding truthfully and according to the trial document. 
    You do not need to repeat parts of the answer if you have already provided the information previously.
    Do not simulate conversations with attorneys, the judge, or other witnesses. Wait until you are directly questioned or called to testify.
    """, 
    llm_config=llm_config, 
    description="""
    Represents the defendant in the mock trial.
    Provides testimony when called to the stand, responds to direct and cross-examination questions, 
    and defends against accusations by presenting their version of events as established in the trial document. 
    The defendant must remain truthful and consistent with the provided case facts while adhering to courtroom protocol.
    """
)

agents["witness_agent"] = ConversableAgent(
    name="witness_agent", 
    system_message="""
    You are a witness in a mock trial, you will represent all the witnesses EXCEPT the defendant, never pretend to be the defendant. 
    When called upon always state your name, and then then answer the question truthfully and consistently.
    Answer questions **only from the relevant witness’s perspective** based on the trial document. 
    When answering questions, only answer one question at a time and end your turn.
    Do not simulate conversations between the witness and other trial participants. Avoid generating content for other roles.
    """, 
    #Only respond **when a specific witness is referenced or called**. 
    #  When called upon always state your name and say you are ready for questioning, and then then answer the question truthfully and consistently.
    llm_config=llm_config, 
    description="""
    Represents all potential witnesses in the mock trial other than defendant.
    Will only speak when a witness other than the defendant is directly referenced or called to testify.
    Follows legal procedures during direct and cross-examinations, providing truthful and consistent testimony in accordance with courtroom protocol.
    """
)

In [918]:
disallowed_transitions = {
    agents["witness_agent"]: [agents["defendant_agent"], agents["witness_agent"]],
    agents["defendant_agent"]: [agents["witness_agent"], agents["defendant_agent"]],
    agents["judge_agent"]: [agents["judge_agent"]],
    agents["human_proxy"]: [agents["human_proxy"]],
}

if user_role == "DA":
    disallowed_transitions[agents["prosecuting_attorney"]] = [agents["prosecuting_attorney"], agents["human_proxy"]]
    disallowed_transitions[agents["human_proxy"]] = [agents["prosecuting_attorney"], agents["human_proxy"]]
else:
    disallowed_transitions[agents["defense_attorney"]] = [agents["defense_attorney"], agents["human_proxy"]]
    disallowed_transitions[agents["human_proxy"]] = [agents["defense_attorney"], agents["human_proxy"]]


group_chat = GroupChat(
    agents=agents.values(),
    messages=[],
    allowed_or_disallowed_speaker_transitions=disallowed_transitions,
    speaker_transitions_type="disallowed",
    max_round=4,
    # allow_repeat_speaker=False,
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config, 
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
)

In [919]:
# initial_message = f"""
# I will be roleplaying as the {human_proxy_role} in this mock trial. I want to start from the direct examination where the defendant is already at the stand and I as the {human_proxy_role} am questioning them.

# Here is the full trial document:

# {pdf_text}
# """

# The prosecuting attorney should only speak after the judge has directly given them the floor, and the defense attorney should only speak after the prosecuting attorney has finished their questioning. The witnesses should only speak when directly addressed by the attorneys.

initial_message = f"""
I will be roleplaying as the {human_proxy_role} in this mock trial. We will begin with the direct examination, where the defendant is already on the stand, and I, as the {human_proxy_role}, will be starting the questioning. However, this simulation will cover the entire court proceeding, including cross-examinations, objections, and any other trial phases.

The prosecuting attorney should only speak after the judge has directly given them the floor, and the defense attorney should only speak after the prosecuting attorney has finished their questioning. The witnesses should only speak when directly addressed by the attorneys.

When an attorney says they are finished with questioning, the next speaker should always be the judge. The judge will then decide who has the floor next based on the courtroom procedure.

For context, the following document contains all the necessary details about the case, including background information, procedural context, and evidence:

<context> {pdf_text} </context>

Please ensure that all responses are appropriate for a courtroom setting, align with the role you are assigned, and adhere to the rules of courtroom procedure.
"""

In [920]:
#initial conversation
chat_result = agents["human_proxy"].initiate_chat(
    group_chat_manager,
    message=initial_message,
    summary_method="reflection_with_llm",
)

conversation_history = []


for i in range(5):
    
    # prev_chat = group_chat_manager.messages_from_string(chat_result.chat_history)
    prev_chat = chat_result.chat_history
    conversation_history.extend(chat_result.chat_history)
    last_agent, last_message = group_chat_manager.resume(messages = prev_chat, remove_termination_string="TERMINATE")

    next_user_input = input("How would you like to reply?")
    if next_user_input.upper() == "STOP":
        break

    # next_message = f"""
    # I have been roleplaying as the {human_proxy_role} in this mock trial. My next question is {next_user_input}
    # """

    chat_result = agents["human_proxy"].initiate_chat(
        group_chat_manager,
        message = next_user_input,
        clear_history = False,
    )

human_proxy-defense_attorney (to chat_manager):


I will be roleplaying as the defense attorney in this mock trial. We will begin with the direct examination, where the defendant is already on the stand, and I, as the defense attorney, will be starting the questioning. However, this simulation will cover the entire court proceeding, including cross-examinations, objections, and any other trial phases.

The prosecuting attorney should only speak after the judge has directly given them the floor, and the defense attorney should only speak after the prosecuting attorney has finished their questioning. The witnesses should only speak when directly addressed by the attorneys.

When an attorney says they are finished with questioning, the next speaker should always be the judge. The judge will then decide who has the floor next based on the courtroom procedure.

For context, the following document contains all the necessary details about the case, including background information, procedural

In [921]:
print(conversation_history)

[{'content': '\nI will be roleplaying as the defense attorney in this mock trial. We will begin with the direct examination, where the defendant is already on the stand, and I, as the defense attorney, will be starting the questioning. However, this simulation will cover the entire court proceeding, including cross-examinations, objections, and any other trial phases.\n\nThe prosecuting attorney should only speak after the judge has directly given them the floor, and the defense attorney should only speak after the prosecuting attorney has finished their questioning. The witnesses should only speak when directly addressed by the attorneys.\n\nWhen an attorney says they are finished with questioning, the next speaker should always be the judge. The judge will then decide who has the floor next based on the courtroom procedure.\n\nFor context, the following document contains all the necessary details about the case, including background information, procedural context, and evidence:\n\n<

In [922]:
convo_string = group_chat_manager.messages_to_string(conversation_history)
print(type(convo_string))

<class 'str'>


In [923]:
feedback_agent = ConversableAgent(
    name="feedback_agent", 
    system_message=f"""
    You are a legal feedback assistant for a mock trial simulation.
    Cite specific examples from the user's performance and provide constructive feedback on their courtroom performance, legal arguments, and trial strategy.
    """,
    llm_config=llm_config, 
    # chat_messages=conversation_history,
    description="""
    Provides detailed legal feedback on courtroom performance, including argument clarity, legal reasoning, 
    presentation style, and strategy based on the entire conversation history.
    """
)

human_agent = ConversableAgent(
        "human_agent",
        llm_config=False,
        human_input_mode="ALWAYS",
        description="""
        Human user asking questions to the feedback agent.
        """
    )

In [924]:
feedback_result = human_agent.initiate_chat(
    recipient=feedback_agent,
    message=f"Please provide me with feedback on how I can improve my courtroom performance, legal arguments, and strategy. I was roleplaying as the {human_proxy_role} in the mock trial. Be specific to this specific case. Here is the entire conversation history: {convo_string}",
)


human_agent (to feedback_agent):

Please provide me with feedback on how I can improve my courtroom performance, legal arguments, and strategy. I was roleplaying as the defense attorney in the mock trial. Be specific to this specific case. Here is the entire conversation history: [{"content": "\nI will be roleplaying as the defense attorney in this mock trial. We will begin with the direct examination, where the defendant is already on the stand, and I, as the defense attorney, will be starting the questioning. However, this simulation will cover the entire court proceeding, including cross-examinations, objections, and any other trial phases.\n\nThe prosecuting attorney should only speak after the judge has directly given them the floor, and the defense attorney should only speak after the prosecuting attorney has finished their questioning. The witnesses should only speak when directly addressed by the attorneys.\n\nWhen an attorney says they are finished with questioning, the next s

KeyboardInterrupt: 